# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns={'Population (2016 est.)[8]':'Population'})
def nhl_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # get rid of all rows that contains '—',
    cities_nhl = cities[['Metropolitan area', 'Population', 'NHL']]
    cities_nhl = cities_nhl[cities_nhl['NHL']!= '—']

    # get rid of these values because they will need to get broken up on column 'NHL',
    cities_nhl = cities_nhl[cities_nhl['Metropolitan area']!= 'New York City']
    cities_nhl = cities_nhl[cities_nhl['Metropolitan area']!= 'Los Angeles']

    # append these key values onto the current dataframe
    a = [{'Metropolitan area': 'New York City', 'Population': 20153634, 'NHL': 'Rangers'},
        {'Metropolitan area': 'New York City', 'Population': 20153634, 'NHL': 'Islanders'},
        {'Metropolitan area': 'New York City', 'Population': 20153634, 'NHL': 'Devils'}]

    b = [{'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NHL': 'Kings'},
        {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NHL': 'Ducks'}]

    a = pd.DataFrame(a)
    b = pd.DataFrame(b)


    # remove these values in NHL; games to be played occurred in 2018
    list_nhl_remove = ['[note 32]', '[note 34]', '[note 39]', '[note 25]', '[note 71]']
    cities_nhl = cities_nhl.set_index('NHL')
    cities_nhl = cities_nhl.drop(list_nhl_remove, axis = 0)

    # reset the index
    cities_nhl = cities_nhl.reset_index()

    # concatenate to get the list of of NHL teams
    cities_nhl = pd.concat([cities_nhl, a, b])

    cities_nhl = cities_nhl[['Metropolitan area', 'Population', 'NHL']]

    # let us replace values that contain [note #] with blank
    cities_nhl['NHL'] = cities_nhl['NHL'].str.replace(r'\[.*\]', '', regex = True)

    # rename column "NHL" to "team"
    cities_nhl = cities_nhl.rename(columns = {'NHL': 'team'})

    cities_nhl = cities_nhl.reset_index()[['Metropolitan area', 'Population', 'team']]
    #######################################################################################################################################################
    # read in the nhl file
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018]

    list_to_remove = ['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division']

    nhl_df = nhl_df.set_index('team')
    
    # get rid of rows that 'Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division' within index 'team'
    nhl_df = nhl_df.drop(list_to_remove, axis = 0)
    nhl_df = nhl_df.reset_index()
    nhl_df = nhl_df.sort_values('team')
    nhl_df['name'] = nhl_df['team'].str.replace(r'\*','', regex = True)

    # get the name of the team by look at the last value on column name
    nhl_df[['t1', 't2','t3']]= nhl_df['name'].str.split(' ', expand = True)

    def updating_name(x):
        if x == 'Blue':
            return 'Blue Jackets'
        elif x == 'Red':
            return 'Red Wings'
        elif x == 'Golden':
            return 'Golden Knights'
        elif x == 'Maple':
            return 'Maple Leafs'
        else:
            return x

    nhl_df['t4'] = nhl_df['t2'].apply(lambda x: updating_name(x))

    # set index
    nhl_df = nhl_df.set_index('t3')

    # the value of the cell based on index and column
    nhl_df['t4']['Islanders'] = 'Islanders'
    nhl_df['t4']['Rangers'] = 'Rangers'
    nhl_df['t4']['Sharks'] = 'Sharks'
    nhl_df['t4']['Lightning'] = 'Lightning'
    nhl_df['t4']['Kings'] = 'Kings'
    nhl_df['t4']['Blues'] = 'Blues'
    nhl_df['t4']['Devils'] = 'Devils'

    nhl_df = nhl_df.rename(columns={'t4':'nhl_team'})

    nhl_df = nhl_df.reset_index()

    nhl_df['sum_win_loss'] = pd.to_numeric(nhl_df['W']) + pd.to_numeric(nhl_df['L'])
    nhl_df['win_loss_ratio'] = pd.to_numeric(nhl_df['W']) / nhl_df['sum_win_loss']

    nhl_df = nhl_df[['nhl_team', 'win_loss_ratio']]
    nhl_df

    # merge the data
    data_merge = pd.merge(cities_nhl, nhl_df, left_on = 'team', right_on = 'nhl_team', how = 'outer')
    data_merge['Population'] = pd.to_numeric(data_merge['Population'])
    data_merge = data_merge.groupby('Metropolitan area').agg({'win_loss_ratio': np.nanmean,
                                                             'Population': np.nanmean})    
    
    population_by_region = data_merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_merge['win_loss_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities.rename(columns={'Population (2016 est.)[8]':'Population'})

def nba_correlation():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    cities_nba = cities[['Metropolitan area', 'Population', 'NBA']]
    
    # get rid values that are '—' in column 'NBA'
    cities_nba = cities_nba[cities_nba['NBA'] != '—']

    list_to_remove = ['[note 27]', '[note 33]', '[note 36]', '[note 38]', '[note 42]', '[note 47]', '[note 58]', '[note 60]', '[note 63]']

    cities_nba = cities_nba.set_index('NBA')

    cities_nba = cities_nba.drop(list_to_remove, axis = 0)
    cities_nba = cities_nba.reset_index()
    cities_nba['team'] = cities_nba['NBA'].str.replace(r'\[.*\]', '', regex = True)
    cities_nba = cities_nba.drop('NBA', axis = 1)

    # we need to append these data to the original data frame; make a copy of these data
    nba_team_add = [{'Metropolitan area': 'New York City', 'Population': 20153634, 'team': 'Knicks'},
             {'Metropolitan area': 'New York City', 'Population': 20153634, 'team': 'Nets'},
             {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'team': 'Lakers'},
               {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'team': 'Clippers'}]

    nba_team_add = pd.DataFrame(nba_team_add)

    # vertically join the two data frames
    cities_nba = pd.concat([cities_nba, nba_team_add])
    cities_nba = cities_nba.set_index('team')

    cities_nba = cities_nba.drop('Lakers Clippers', axis = 'index')
    cities_nba = cities_nba.drop('Knicks Nets', axis = 'index')
    cities_nba = cities_nba.reset_index()
    cities_nba = cities_nba[['Metropolitan area', 'Population', 'team']]

    cities_nba['team'] = cities_nba['team'].str.strip()
    ####################################################################################################################################
    # clean up the nba_df
    nba_dataframe = nba_df
    nba_dataframe = nba_dataframe[nba_dataframe['year'] == 2018]
    nba_dataframe = nba_dataframe[['team', 'W', 'L']]

    # clean up data with the character of '*' and every alpha, numeric, and puncation after
    nba_dataframe['team2'] = nba_dataframe['team'].str.replace(r'\*.*', '', regex = True)

    # clean up characters with parathesis and everything after
    nba_dataframe['team2'] = nba_dataframe['team2'].str.replace(r'[(]\d*[)]', '', regex = True)

    # break up the dataset
    # get the name of the team by look at the last value on column name
    nba_dataframe[['split_name0', 'split_name1', 'split_name2']]= nba_dataframe['team2'].str.split(' ', expand = True)

    # be sure to remove leading and trailing 
    nba_dataframe['split_name2'] = nba_dataframe['split_name2'].str.strip()
    nba_dataframe['split_name1'] = nba_dataframe['split_name1'].str.strip()

    # we want to up column 'split_name1'
    nba_dataframe = nba_dataframe.set_index('split_name2')
    nba_dataframe['split_name1']['Blazers'] = 'Trail Blazers'

    nba_dataframe['split_name1']['Thunder'] = 'Thunder'
    nba_dataframe['split_name1']['Clippers'] = 'Clippers'
    nba_dataframe['split_name1']['Lakers'] = 'Lakers'
    nba_dataframe['split_name1']['Knicks'] = 'Knicks'
    nba_dataframe['split_name1']['Pelicans'] = 'Pelicans'
    nba_dataframe['split_name1']['Warriors'] = 'Warriors'
    nba_dataframe['split_name1']['Spurs'] = 'Spurs'

    nba_dataframe = nba_dataframe.reset_index()
    nba_dataframe['win_loss'] = pd.to_numeric(nba_dataframe['W']) +  pd.to_numeric(nba_dataframe['L'])
    nba_dataframe['win_loss_ratio'] = pd.to_numeric(nba_dataframe['W']) / nba_dataframe['win_loss']
    
    # make selection
    nba_dataframe = nba_dataframe[['split_name1', 'win_loss_ratio']]
    nba_dataframe = nba_dataframe.rename(columns = {'split_name1': 'nba_team'})
    
    # merge the dataframe
    data_merge = pd.merge(cities_nba, nba_dataframe, left_on ='team', right_on = 'nba_team', how = 'outer')
    
    # convert column 'Population' to numeric
    data_merge['Population'] = pd.to_numeric(data_merge['Population'])
    
    # use groupby and apply
    answer = data_merge.groupby('Metropolitan area').agg({'Population': np.nanmean,
                                                         'win_loss_ratio':np.nanmean})

    population_by_region = answer['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = answer['win_loss_ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns={'Population (2016 est.)[8]':'Population'})

def mlb_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()

    cities_mlb = cities[['Metropolitan area', 'Population', 'MLB']]
    cities_mlb = cities_mlb[cities_mlb['MLB'] != '—']
    cities_mlb['MLB'] = cities_mlb['MLB'].str.replace(r'\[.*\]', '', regex = True)

    # filtered out empty string
    cities_mlb = cities_mlb.drop([25, 29, 30], axis = 0)
    cities_mlb = cities_mlb.reset_index()

    cities_mlb = cities_mlb.drop('index', axis = 1)
    
        # create a list which will added to the data frame 'cities_mlb'
    mlb_list = [
               {'Metropolitan area': 'New York City', 'Population': 20153634, 'MLB': 'Yankees'},
               {'Metropolitan area': 'New York City', 'Population': 20153634, 'MLB': 'Mets'},
               {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'MLB': 'Dodgers'},
               {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'MLB': 'Angels'},
               {'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'MLB': 'Giants'},
               {'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'MLB': 'Athletics'},
               {'Metropolitan area': 'Chicago', 'Population': 9512999, 'MLB': 'Cubs'},
               {'Metropolitan area': 'Chicago', 'Population': 9512999, 'MLB': 'White Sox'},
               ]
    mlb_list = pd.DataFrame(mlb_list)

    # concatenate the data frames
    cities_mlb = pd.concat([cities_mlb, mlb_list])

    cities_mlb = cities_mlb.set_index('MLB')
    drop_list = ['Yankees Mets', 'Dodgers Angels', 'Giants Athletics', 'Cubs White Sox']
    cities_mlb = cities_mlb.drop(drop_list, axis = 0)
    cities_mlb = cities_mlb.reset_index()
    cities_mlb = cities_mlb[['Metropolitan area', 'Population', 'MLB']]
    
    # delete trailing and leading whitespace on column 'MLB' before merging
    cities_mlb['MLB'] = cities_mlb['MLB'].str.strip()
    ##########################################################################################################
    # clean mlb_df
    mlb_dataframe = mlb_df
    mlb_dataframe=mlb_dataframe[mlb_dataframe['year'] == 2018]
    mlb_dataframe[['s0', 's1','s2']] = mlb_dataframe['team'].str.split(' ', expand = True)
    
    mlb_dataframe['s1'] = mlb_dataframe['s1'] .str.strip()
    mlb_dataframe['s2'] = mlb_dataframe['s2'] .str.strip()
    
    # set index on column 's2'
    mlb_dataframe = mlb_dataframe.set_index('s2')

    # update values on column 's1'
    mlb_dataframe['s1']['Sox'] = 'Red Sox'
    mlb_dataframe['s1']['Rays'] = 'Rays'
    mlb_dataframe['s1']['Jays'] =  'Blue Jays'
    mlb_dataframe['s1']['Angels'] = 'Angels'
    mlb_dataframe['s1']['Cardinals'] = 'Cardinals'
    mlb_dataframe['s1']['Dodgers'] = 'Dodgers'
    mlb_dataframe['s1']['Giants'] = 'Giants'
    mlb_dataframe['s1']['Padres'] = 'Padres'
    mlb_dataframe['s1']['Yankees'] = 'Yankees'
    mlb_dataframe['s1']['Royals'] = 'Royals'
    mlb_dataframe['s1']['Mets'] = 'Mets'
    
    mlb_dataframe = mlb_dataframe.reset_index()
    mlb_dataframe['s1'][8] = 'White Sox'
 
    mlb_dataframe = mlb_dataframe.rename(columns = {'s1': 'mlb_team'})
    mlb_dataframe['win_loss'] = pd.to_numeric(mlb_dataframe['W']) + pd.to_numeric(mlb_dataframe['L'])
    mlb_dataframe['win_loss_ratio'] =pd.to_numeric(mlb_dataframe['W']) / mlb_dataframe['win_loss']

    mlb_dataframe = mlb_dataframe[['mlb_team', 'win_loss_ratio']]
    
    # merge the data
    data_merge = pd.merge(cities_mlb, mlb_dataframe, left_on = 'MLB', right_on = 'mlb_team', how = 'outer')
    data_merge['Population'] = pd.to_numeric(data_merge['Population'])    
    ##########################################################################################################
    answer = data_merge.groupby('Metropolitan area').agg({'Population': np.nanmean, 
                                                      'win_loss_ratio': np.nanmean})
    
    population_by_region = answer['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = answer['win_loss_ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
  

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.rename(columns={'Population (2016 est.)[8]':'Population'})

def nfl_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()
    cities_nfl = cities[['Metropolitan area', 'Population', 'NFL']]
    cities_nfl = cities_nfl[cities_nfl['NFL'] != '—']
    cities_nfl['NFL'] = cities_nfl['NFL'].str.replace(r'\[.*\]', '', regex = True)

    # drop these rows
    cities_nfl['NFL'] = cities_nfl['NFL'].replace('', np.nan)
    cities_nfl = cities_nfl.dropna(subset =['NFL'])

    cities_nfl = cities_nfl.drop(13, axis = 0)

    cities_nfl = cities_nfl.reset_index()
    cities_nfl = cities_nfl.drop('index', axis = 1)

    # add these values
    nfl_add = [{'Metropolitan area': 'New York City', 'Population': 20153634, 'NFL': 'Giants'},
              {'Metropolitan area': 'New York City', 'Population': 20153634, 'NFL': 'Jets'},
              {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NFL': 'Rams'},
              {'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NFL': 'Chargers'},
              {'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'NFL': '49ers'},
              {'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'NFL': 'Raiders'}]

    # convert the created list of dictionaries into a data frame
    nfl_add = pd.DataFrame(nfl_add)

    # vertically join the two data frames; use the concat function
    cities_nfl = pd.concat([cities_nfl, nfl_add])

    list_to_remove = ['Giants Jets', 'Rams Chargers', '49ers Raiders']

    cities_nfl = cities_nfl.set_index('NFL')
    cities_nfl = cities_nfl.drop(list_to_remove, axis = 0)
    cities_nfl = cities_nfl.reset_index()
    cities_nfl = cities_nfl[['Metropolitan area', 'Population', 'NFL']]

    # clean up the nfl_df
    nfl_dataframe = nfl_df[nfl_df['year'] == 2018]

    list_to_remove = ['AFC East', 'AFC North', 'AFC West', 'AFC South', 'NFC East', 'NFC North', 'NFC West', 'NFC South']

    nfl_dataframe = nfl_dataframe.set_index('team')
    nfl_dataframe = nfl_dataframe.drop(list_to_remove, axis = 0)
    nfl_dataframe = nfl_dataframe.reset_index()

    nfl_dataframe['team'] = nfl_dataframe['team'].str.replace(r'\*','', regex = True)
    nfl_dataframe['team'] = nfl_dataframe['team'].str.replace(r'\+','', regex = True)

    nfl_dataframe = nfl_dataframe[['team', 'W', 'L']]
    nfl_dataframe[['s0', 's1', 's2']] = nfl_dataframe['team'].str.split(' ', expand = True)

    # set index
    nfl_dataframe = nfl_dataframe.set_index('s2')

    # update values
    nfl_dataframe['s1']['Patriots'] = 'Patriots'
    nfl_dataframe['s1']['Chiefs'] = 'Chiefs'
    nfl_dataframe['s1']['Chargers'] = 'Chargers'
    nfl_dataframe['s1']['Packers'] = 'Packers'
    nfl_dataframe['s1']['Buccaneers'] = 'Buccaneers'
    nfl_dataframe['s1']['Rams'] = 'Rams'
    nfl_dataframe['s1']['Jets'] = 'Jets'
    nfl_dataframe['s1']['Giants'] = 'Giants'
    nfl_dataframe['s1']['49ers'] = '49ers'
    nfl_dataframe['s1']['Saints'] = 'Saints'

    # reset index
    nfl_dataframe = nfl_dataframe.reset_index()
    nfl_dataframe = nfl_dataframe.rename(columns = {'s1': 'team_nfl'})

    nfl_dataframe['win_loss'] = pd.to_numeric(nfl_dataframe['W']) + pd.to_numeric(nfl_dataframe['L']) 
    nfl_dataframe['win_loss_ratio'] = pd.to_numeric(nfl_dataframe['W']) / nfl_dataframe['win_loss']

    nfl_dataframe = nfl_dataframe[['team_nfl', 'win_loss_ratio']]

    # merge the data frames
    data_merge = pd.merge(cities_nfl, nfl_dataframe, left_on = 'NFL', right_on = 'team_nfl', how = 'outer')
    data_merge = data_merge[['Metropolitan area', 'Population', 'win_loss_ratio']]
    data_merge['Population'] = pd.to_numeric(data_merge['Population'])

    # use the functions groupby and agg
    data_merge = data_merge.groupby('Metropolitan area').agg({'Population': np.nanmean,
                                                             'win_loss_ratio': np.nanmean})

    
    population_by_region = data_merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data_merge['win_loss_ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values